### Integração Numérica - Regra do Trapézio

Este notebook apresenta a implementação da Regra do Trapézio para cálculo de integrais definidas, incluindo o erro de aproximação simples e o erro generalizado.

In [2]:
%load_ext autoreload
%autoreload 2
from integral import Integral
import math
import numpy as np

### Classe `IntegralTrapezio`

In [3]:
class IntegralTrapezio(Integral):
    def __init__(self, x=None, y=None, limite_inf=None, limite_sup=None, func_integral=None, h=None):
        super().__init__()
        self._setup(x, y, limite_inf, limite_sup, func_integral, h)

    def integral(self):
        # regra do trapézio simples sobre todos os pontos
        soma = self.y[0] + self.y[-1] + 2 * np.sum(self.y[1:-1])
        return (self.h / 2) * soma

    def erro_simples(self):
        d2_inf = self.interpolador.derivative(self.limite_inf, 2)
        d2_sup = self.interpolador.derivative(self.limite_sup, 2)
        maior = self.saber_maior(d2_inf, d2_sup)
        parcela_1 = ((self.limite_sup - self.limite_inf) * (self.h**2)) / 12
        return abs(maior * parcela_1)

    def erro_generalizado(self):
        # No trapézio geralmente erro é dado pela segunda derivada (2ª ordem)
        return self.erro_simples()

### Exemplo de uso com pontos

In [4]:
# Questão velocidade - lista
def f(x):
    return x * math.sqrt(x**2 + 1)

pontos_x = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
pontos_y = [0, 50.4, 53.7, 67.5, 74.3, 84.6, 92.1, 98.3, 100, 105, 110]

trap = IntegralTrapezio(x=pontos_x, y=pontos_y)
print(f"Trapézio (x, y): {trap.integral():.5f}")
print(f"Erro simples: {trap.erro_simples():.5f}")
print(f"Erro generalizado: {trap.erro_generalizado():.5f}")


Trapézio (x, y): 780.90000
Erro simples: 1237.71209
Erro generalizado: 1237.71209


In [10]:
# Questão margens do rio - lista

pontos_x = [30, 50]
pontos_y = [171.2, 95.3]

trap = IntegralTrapezio(x=pontos_x, y=pontos_y)
print(f"Trapézio parte 2 da M2 (x, y): {trap.integral():.5f}")
print(f"Erro simples: {trap.erro_simples():.5f}")
print(f"Erro generalizado: {trap.erro_generalizado():.5f}")

pontos_x = [30, 50]
pontos_y = [72.8, 51]

trap = IntegralTrapezio(x=pontos_x, y=pontos_y)
print(f"Trapézio parte 2 da M1 (x, y): {trap.integral():.5f}")
print(f"Erro simples: {trap.erro_simples():.5f}")
print(f"Erro generalizado: {trap.erro_generalizado():.5f}")

Trapézio parte 2 da M2 (x, y): 2665.00000
Erro simples: 0.00000
Erro generalizado: 0.00000
Trapézio parte 2 da M1 (x, y): 1238.00000
Erro simples: 0.00000
Erro generalizado: 0.00000


### Exemplo de uso com função

In [9]:
# Questão 2: integral de 0 a 1 3*exp(-x) - Lista
def f(x):
    return 3 * math.exp(-x)

trap2 = IntegralTrapezio(limite_inf=0, limite_sup=1, func_integral=f, h=1/5)
print(f"Trapézio (x, y): {trap2.integral():.4f}")
print(f"Erro simples: {trap2.erro_simples():.4f}")
print(f"Erro generalizado: {trap2.erro_generalizado():.4f}")


Trapézio (x, y): 1.9027
Erro simples: 0.0100
Erro generalizado: 0.0100
